In [ ]:
""" Video - Mdeals in the Summer Olympics"""

In [ ]:
Question: Does a country win more Olympic Medals when it is the HOST country

In [1]:
#Ex1 -prepare the DF from a TSV

#Import pandas
import pandas as pd

# Create file path: file_path
file_path = 'Summer Olympic medalists 1896 to 2008 - EDITIONS.tsv'

# Load DataFrame from file_path: editions
editions = pd.read_csv(file_path, sep='\t') #notice the sep flag used for .tsv

# Extract the relevant columns: editions
editions = editions[['Edition', 'Grand Total', 'City', 'Country']]

# Print editions DataFrame
print(editions)

    Edition  Grand Total         City                     Country
0      1896          151       Athens                      Greece
1      1900          512        Paris                      France
2      1904          470    St. Louis               United States
3      1908          804       London              United Kingdom
4      1912          885    Stockholm                      Sweden
5      1920         1298      Antwerp                     Belgium
6      1924          884        Paris                      France
7      1928          710    Amsterdam                 Netherlands
8      1932          615  Los Angeles               United States
9      1936          875       Berlin                     Germany
10     1948          814       London              United Kingdom
11     1952          889     Helsinki                     Finland
12     1956          885    Melbourne                   Australia
13     1960          882         Rome                       Italy
14     196

In [3]:
#Ex2 - prepare a DF from csv file
# Create the file path: file_path
file_path = 'Summer Olympic medalists 1896 to 2008 - IOC COUNTRY CODES.csv'

# Load DataFrame from file_path: ioc_codes
ioc_codes = pd.read_csv(file_path)

# Extract the relevant columns: ioc_codes
ioc_codes = ioc_codes[['Country', 'NOC']]

# Print first and last 5 rows of ioc_codes
print(ioc_codes.head())
print(ioc_codes.tail())

           Country  NOC
0      Afghanistan  AFG
1          Albania  ALB
2          Algeria  ALG
3  American Samoa*  ASA
4          Andorra  AND
             Country  NOC
196          Vietnam  VIE
197  Virgin Islands*  ISV
198            Yemen  YEM
199           Zambia  ZAM
200         Zimbabwe  ZIM


In [4]:
#Ex3 - building medals DataFrame
#Goal - build a dict with olympic edition years as keys and DFs as values
#the dict is built inside a loop over year

# Create empty dictionary: medals_dict
medals_dict = {}

#note, this loop most likely won't work in my dev env because the dataset I got
#from DataCamp do not have these following csv...
for year in editions['Edition']:

    # Create the file path: file_path
    file_path = 'summer_{:d}.csv'.format(year)
    
    # Load file_path into a DataFrame: medals_dict[year]
    medals_dict[year] = pd.read_csv(file_path)
    
    #Select only the columns 'Athlete', 'NOC', and 'Medal' from medals_dict[year].
    #medals_dict[year] = [v for v in medals_dict.items() if (v == 'Athlete') or (v =='NOC') or (v=='Medal')] #transforms to list deprecate
    
    medals_dict[year] = medals_dict[year][['Athlete', 'NOC', 'Medal']]

    #print(type(medals_dict[year])) #confirmed it retains DataFrame type

    
    # Assign year to column 'Edition' of medals_dict
    
    #create a new col called 'Edition' whose entries are all year
    #should I do a broadcast here?
    medals_dict[year]['Edition'] = year
    
# Concatenate medals_dict: medals
medals = pd.concat(medals_dict, ignore_index=True)

# Print first and last 5 rows of medals
print(medals.head())
print(medals.tail())

FileNotFoundError: [Errno 2] File b'summer_1896.csv' does not exist: b'summer_1896.csv'

In [ ]:
#Ex4 - Counting medals by country/edition in a pivot
import pandas as pd
# Construct the pivot_table: medal_counts
medal_counts = pd.pivot_table(medals,
                              index='Edition',
                              values='Athlete',
                              columns='NOC',
                              aggfunc='count')

# Print the first & last 5 rows of medal_counts
print(medal_counts.head())
print(medal_counts.tail())

In [ ]:
#Ex5 - Computing fractions of medals per Olympic edition
# Set Index of editions: totals
totals = editions.set_index('Edition')

# Reassign totals['Grand Total']: totals
totals = totals['Grand Total']

# Divide medal_counts by totals: fractions
fractions = medal_counts.divide(totals, axis='rows')

# Print first & last 5 rows of fractions
print(fractions.head())
print(fractions.tail())

In [ ]:
#Ex6 - Computing percentage change in fraction of medals won
#Goal: see if there is a host country advantage means need to see how fractiosn of mdeals won
#changes from edition to edition
#Note will use expanding mean - value of the mean of all the data available up to that point in time

# Apply the expanding mean: mean_fractions
mean_fractions = fractions.expanding().mean()

# Compute the percentage change: fractions_change
fractions_change = mean_fractions.pct_change() * 100

# Reset the index of fractions_change: fractions_change
fractions_change = fractions_change.reset_index()

# Print first & last 5 rows of fractions_change
print(fractions_change.head())
print(fractions_change.tail())

In [ ]:
#Ex7 - prepare a DF 'hosts' by left joining editions and ioc_codes
#subset Edition and NOC columns and set Edition as Index
#deal with missing NOC values
#Reset Index and print the final DF

# Import pandas
import pandas as pd

# Left join editions and ioc_codes: hosts
hosts = pd.merge(editions, ioc_codes, how='left')

# Extract relevant columns and set index: hosts
hosts = hosts[['Edition', 'NOC']].set_index('Edition')

# Fix missing 'NOC' values of hosts, next 4 lines were given to me by DataCamp
print(hosts.loc[hosts.NOC.isnull()])
hosts.loc[1972, 'NOC'] = 'FRG'
hosts.loc[1980, 'NOC'] = 'URS'
hosts.loc[1988, 'NOC'] = 'KOR'

# Reset Index of hosts: hosts
hosts = hosts.reset_index()

# Print hosts
print(hosts)

In [ ]:
#Ex8 - reshaping for analysis
# Import pandas
import pandas as pd

# Reshape fractions_change: reshaped
reshaped = pd.melt(fractions_change,
                   id_vars='Edition',
                   value_name='Change')

# Print reshaped.shape and fractions_change.shape
print(reshaped.shape, fractions_change.shape)

# Extract rows from reshaped where 'NOC' == 'CHN': chn
chn = reshaped.loc[reshaped['NOC'] == 'CHN'] #learned this Merging DF Ch2 - Ex4

# Print last 5 rows of chn with .tail()
print(chn.tail())

In [ ]:
#Ex9 - Merging to compute influence
#merge DF reshaped with hosts and tidy results
#End result is a DF summarizing the fractional change in the expanding mean of the percentage of medals
#won for the host country in each Olympic edition

# Import pandas
import pandas as pd

# Merge reshaped and hosts: merged
merged = pd.merge(reshaped, hosts, how='inner')

# Print first 5 rows of merged
print(merged.head())

# Set Index of merged and sort it: influence
influence = merged.set_index('Edition').sort_index()

# Print first 5 rows of influence
print(influence.head())

In [ ]:
#Ex10 - Plot the influence of being a host country
# Import pyplot
import matplotlib.pyplot as plt

# Extract influence['Change']: change
change = influence['Change']

# Make bar plot of change: ax
ax = change.plot(kind='bar')

# Customize the plot to improve readability
ax.set_ylabel("% Change of Host Country Medal Count")
ax.set_title("Is there a Host Country Advantage?")
ax.set_xticklabels(editions['City'])

# Display the plot
plt.show()